## Machine Leaning 실습

### Classification With Python

In [1]:
%config Completer.use_jedi = False

In [2]:
# 1. package 가져오기

import pandas as pd
import pandas_profiling
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
#classification(분류)
from sklearn.tree import DecisionTreeClassifier      #의사결정트리
from sklearn.ensemble import RandomForestClassifier  #랜덤포레스트
from sklearn.neighbors import KNeighborsClassifier   #KNN(K_Nearst_Neighbor)K-최근접이웃
from sklearn.linear_model import LogisticRegression  #로지스틱회귀
from sklearn.svm import SVC


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score

ads = pd.read_csv('../202106_MachineLearningClass/02Social_Network_Ads.csv')


In [3]:
# 2. 데이터 프레임
ads

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15566689,Female,35.0,57000.0,0
1,15569641,Female,58.0,95000.0,1
2,15570769,Female,26.0,80000.0,0
3,15570932,Male,34.0,115000.0,0
4,15571059,Female,33.0,41000.0,0
...,...,...,...,...,...
395,15813113,Male,40.0,107000.0,1
396,15814004,Male,27.0,20000.0,0
397,15814553,Male,57.0,60000.0,1
398,15814816,Male,31.0,66000.0,0


In [4]:
# 3. 데이터 살펴보기
ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   User ID          400 non-null    int64  
 1   Gender           394 non-null    object 
 2   Age              390 non-null    float64
 3   EstimatedSalary  388 non-null    float64
 4   Purchased        400 non-null    int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 15.8+ KB


In [5]:
ads.describe(include = 'all')

,User ID,Gender,Age,EstimatedSalary,Purchased
count,4.000000e+02,394,390.000000,388.000000,400.000000
unique,NaN,2,NaN,NaN,NaN
top,NaN,Female,NaN,NaN,NaN
freq,NaN,202,NaN,NaN,NaN
mean,1.569154e+07,NaN,37.782051,69628.865979,0.357500
std,7.165832e+04,NaN,10.452300,33889.337949,0.479864
min,1.556669e+07,NaN,18.000000,15000.000000,0.000000
25%,1.562676e+07,NaN,30.000000,43000.000000,0.000000
50%,1.569434e+07,NaN,37.000000,70000.000000,0.000000
75%,1.575036e+07,NaN,46.000000,87250.000000,1.000000


In [6]:
# 4. feature/label 분리
feature = ads.drop(['User ID','Purchased'], axis=1)
label = ads[['Purchased']]

mean_imp = SimpleImputer(strategy='mean')

mean_imp.fit(feature.iloc[:,1:])
feature.iloc[:,1:] = mean_imp.transform(feature.iloc[:,1:])

feature.isnull().sum()

In [7]:
# 5. 빠진 값 확인 
feature.isnull().sum()

Gender              6
Age                10
EstimatedSalary    12
dtype: int64

In [8]:
# 6. Clean Missing Data 1 - numeric

#ads.dropna(axis=0, inplace=True)


In [9]:
# 6-1 . Clean Missing Data 1 - numeric(Age, EstimatedSalary) -mean
from sklearn.impute import SimpleImputer

mean_imputer = SimpleImputer(strategy = 'mean')

def f_mean_imputer(df, i):
    mean_imputer.fit(feature.iloc[:, i:]) # 각 열의 평균으로 대체
    feature.iloc[:, i:] = mean_imputer.transform(df.iloc[:, i:])
    return df.isnull().sum()

# 6-2. Clean Missing Data 2 - string(Gender) -most_frequent
mode_imputer = SimpleImputer(strategy = 'most_frequent')
def f_mode_imputer(df, i):
    mode_imputer.fit(feature.iloc[:, i:]) # 각 열의 최빈값으로 대체
    feature.iloc[:, i:] = mode_imputer.transform(df.iloc[:, i:])
    return df.isnull().sum()

In [10]:
#Age, EstimatedSalary 결측값 평균값으로 대체
f_mean_imputer(feature, 1)
#gender는 최빈값으로 대체
f_mode_imputer(feature, 0)

Gender             0
Age                0
EstimatedSalary    0
dtype: int64

In [12]:
# 7. One hot encoding
# gender 문자열이니까 숫자로 바꿔주기
ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(),[0])], remainder='passthrough')

feature = ct.fit_transform(feature)

print(feature)

[[1.00e+00 0.00e+00 3.50e+01 5.70e+04]
 [1.00e+00 0.00e+00 5.80e+01 9.50e+04]
 [1.00e+00 0.00e+00 2.60e+01 8.00e+04]
 ...
 [0.00e+00 1.00e+00 5.70e+01 6.00e+04]
 [0.00e+00 1.00e+00 3.10e+01 6.60e+04]
 [1.00e+00 0.00e+00 4.50e+01 1.31e+05]]


In [13]:
# 8. split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature, label, test_size=0.2, random_state=142)

print(X_train)
print('-'*77)
print(y_train)

[[0.00e+00 1.00e+00 3.00e+01 1.70e+04]
 [0.00e+00 1.00e+00 3.60e+01 9.90e+04]
 [1.00e+00 0.00e+00 3.20e+01 1.50e+05]
 ...
 [0.00e+00 1.00e+00 5.90e+01 1.43e+05]
 [1.00e+00 0.00e+00 5.70e+01 2.60e+04]
 [1.00e+00 0.00e+00 5.80e+01 3.80e+04]]
-----------------------------------------------------------------------------
     Purchased
249          0
213          1
202          1
291          0
144          1
..         ...
173          1
26           1
268          1
197          1
277          1

[320 rows x 1 columns]


In [14]:
# 9. train


dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
log_clf = LogisticRegression()

In [ ]:
dt_clf.fit(X_train, y_train)

In [ ]:
rf_clf.fit(X_train, y_train)

In [ ]:
knn_clf.fit(X_train, y_train)

In [ ]:
svm_clf.fit(X_train, y_train)

In [15]:
log_clf.fit(X_train, y_train)

/anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [16]:
# 10. Score
y_pred = log_clf.predict(X_test)
print(y_pred)
print(y_test)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
     Purchased
211          0
63           0
130          1
201          0
209          0
..         ...
281          0
300          1
189          1
309          0
70           1

[80 rows x 1 columns]


In [17]:
# 11. Evaluate
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)


print('mae = ',mae)
print('mse = ',mse)
print('r2 = ',r2)
print('accuracy = ',acc)
print('precision = ',pre)
print('recall = ',rec)

mae =  0.3625
mse =  0.3625
r2 =  -0.5686274509803924
accuracy =  0.6375
precision =  0.0
recall =  0.0


/anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# 12. Confusion Matrix


In [ ]:
# 13. CM Visualize
